In [ ]:
import numpy as np
import time

In [ ]:
class BadacGauss:
    def __init__(self, X_train, Xerr_train, Y_train):
        self.X_train = X_train
        self.Xerr_train = Xerr_train
        self.Y_train = Y_train
        
        self.class_list =  np.unique(Y_train)
        self.nclasses = self.class_list.shape[0]
    
    def test(self, X_test, Xerr_test, contamination=0):
        self.X_test = X_test
        self.Xerr_test = Xerr_test
        self.contamination = contamination
        
        self.ntest = X_test.shape[0]
        self.prob_matrix = np.zeros(shape=[self.ntest,self.nclasses])
        
        for i in range(self.nclasses):
            self.index = np.where(self.Y_train == self.class_list[i])
            self.prob_matrix[:,i] = ClassI(self.X_test,self.Xerr_test,self.X_train[self.index],
                            self.Xerr_train[self.index],self.ntest).evaluate()
            delattr(self,'index')
    
    def print_log_proba(self):
        return self.prob_matrix


class ClassI(BadacGauss):
    def __init__(self, X_test, Xerr_test, X_train, Xerr_train, ntest):
        self.X_test = X_test
        self.Xerr_test = Xerr_test
        self.X_train = X_train
        self.Xerr_train = Xerr_train
        self.ntest = ntest
    
    def evaluate(self):
        def p_i(di,sigdi,yi,sigyi):
            C1 = sigdi**-2
            C2 = sigyi**-2
            
            p1 = np.log((2*np.pi*(C1+C2))**-0.5 / (sigdi*sigyi))
            p2 = 0.5 * (C1*di**2 + C2*yi**2)
            p3 = 0.5 * ((di*C1 + yi*C2)**2) / (C1 + C2)
            return p1-p2+p3
        
        self.prob_list = np.zeros(self.ntest)
        self.ntrain = self.X_train.shape[0]
        
        for i in range(self.ntest):
            self.p = np.sum(np.exp(np.float128(np.sum(p_i(self.X_test[i,:],self.Xerr_test[i,:],self.X_train,
                                                          self.Xerr_train),axis=1))))
            self.prob_list[i] = np.log(self.p) + np.log(0.5) - np.log(self.ntrain)
            delattr(self,'p')
            
        return self.prob_list

In [ ]:
def bhm(d, sigd, y, sigy, labels):
    """
    Returns the normalised log probability of belonging to each of 2 datasets,
    here type0 and type1.
    
    Input Parameters:
    d: 1D array (observed data points)
    sigd: 1D array (observed errorbars)
    y: 2D array (template data points) - each row represents the i-th data curve
    sigy: 2D array (template errorbars) - each row represents the i-th data curve
    labels: 1D array with element format int (labels for input curves 'd')
    
    Output: 
    P0-sumPi: Total normalised log probability of belonging to type 0
    P1-sumPi: Total normalised log probability of belonging to type 1"""
    
    def p_i(di, sigdi, yi, sigyi):
        """
        Returns the probability of one point being similar to another point
        Input parameters:
        di: datum being measured
        sigdi: datum errorbar
        yi: training datum
        sigyi: training datum errorbar
        ybari: approximated mean for assumed distribution
        Ryi: approximated errorbar for assumed distribution
            
        Output:
        p1-p2+p3: unnormalised log probability"""
        
        C1 = sigdi**-2
        C2 = sigyi**-2
        
        p1 = np.log((2*np.pi*(C1+C2))**-0.5 / (sigdi*sigyi))
        p2 = 0.5 * (C1*di**2 + C2*yi**2)
        p3 = 0.5 * ((di*C1 + yi*C2)**2) / (C1 + C2)
        return p1-p2+p3
    
    """
    #class 0
    """
    index0 = np.where(labels==0)
    y0 = y[index0,:][0]
    sigy0 = sigy[index0,:][0]
    m0,n0 = np.shape(y0) #m number of training curves, n number of datapoints per curve
    
    P0matrix = p_i(d,sigd,y0,sigy0)
    P0array = np.sum(P0matrix,axis=1)    
    
    Ptau0 = 0.5
    P0 = (np.log(np.sum(np.exp(np.float128(P0array))))) + np.log(Ptau0) - np.log(m0)
    
    """
    #class 1
    """
    index1 = np.where(labels==1)
    y1 = y[index1,:][0]
    sigy1 = sigy[index1,:][0]
    m1,n1 = np.shape(y1) #m number of training curves, n number of datapoints per curve
    
    P1matrix = p_i(d,sigd,y1,sigy1)
    P1array = np.sum(P1matrix,axis=1)
    
    Ptau1 = 0.5
    P1 = (np.log(np.sum(np.exp(np.float128(P1array))))) + np.log(Ptau1) - np.log(m1)
    return P0,P1